In [1]:
# Install needed lib
!pip install --upgrade accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
# from trl.utils import get_peft_config

In [3]:
from datasets import Dataset
import re

file_path = os.path.abspath(__file__)
PROJECT_PATH = os.path.dirname(file_path)

oa_dataset = Dataset.from_json(PROJECT_PATH + '/finetune_data/llm/qs_ans.jsonl')

def chat_template(example):
    input_text = example['input']
    output_text = example['output']
    context = example['context']
    
    input_prefix = "[INST]"
    input_suffix = "[/INST]"
    pre_prompt_prefix = "<<SYS>>\n"
    pre_prompt_suffix = "\n<</SYS>>\n\n"

    pre_prompt = (
        "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, "
        "đồng thời giữ an toàn.\nCâu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, "
        "phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời "
        "của bạn không có thiên kiến xã hội và mang tính tích cực. Nếu một câu hỏi không có ý nghĩa hoặc không "
        "hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. "
        "Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trả lời rằng bạn không biết và vui lòng không "
        "chia sẻ thông tin sai lệch."
    )

    # Ghép các phần lại với nhau
    formatted_prompt = (
        f"{input_prefix}{pre_prompt_prefix}{pre_prompt}{pre_prompt_suffix}"
        f"Ngữ cảnh: {context}\n\n"
        f"Câu hỏi: {input_text}\n"
        f"{input_suffix}"
        f"Câu trả lời: {output_text}"
    )
    return {'text': formatted_prompt}

transformed_dataset = oa_dataset.map(chat_template)


Map:   0%|          | 0/12856 [00:00<?, ? examples/s]

In [4]:
model_name = "meta-llama/Llama-3.2-1B"
finetune_model = "llama-1B-finetune-medical-rag"

# Output folder
output_dir = "./llm_results"

# No of epochs
num_train_epochs =1

# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False # To quantization
lora_r, lora_alpha, lora_dropout = 64, 16, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 1, 1
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}


In [5]:
# transformed_dataset is there
import torch

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model, # model se fintune
    train_dataset=transformed_dataset, # Dataset
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
)

<ipython-input-5-8548a90d9472>:59: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/12856 [00:00<?, ? examples/s]

In [7]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
25,2.414000
50,2.437200
75,2.327300
100,2.102500
125,2.274800
150,1.771400
175,2.091000
200,1.429400
225,1.999700
250,1.201600


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6769bc63-197ad7b20e15e33c31c50df0;22a3a0ee-1b37-4001-aac2-65ccac6186dd)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=12856, training_loss=1.5519499244642465, metrics={'train_runtime': 9204.7207, 'train_samples_per_second': 1.397, 'train_steps_per_second': 1.397, 'total_flos': 5.835818670804173e+16, 'train_loss': 1.5519499244642465, 'epoch': 1.0})

In [8]:
# Save trained model
trainer.model.save_pretrained(finetune_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6769bc64-45e0de223f0bf29f7fb84e45;a6d22633-3f66-4bdb-b9c9-43c11f729ced)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B - will assume that the vocabulary was not modified.
  warnings.warn(


In [24]:
# Test model
logging.set_verbosity(logging.CRITICAL)

def chat_template(input_text, context):
    input_prefix = "<s>[INST] "
    input_suffix = "[/INST] "
    pre_prompt_prefix = "<<SYS>>\n"
    pre_prompt_suffix = "<</SYS>> \n\n"

    pre_prompt = (
        "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, "
        "đồng thời giữ an toàn.\nCâu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, "
        "phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời "
        "của bạn không có thiên kiến xã hội và mang tính tích cực. Nếu một câu hỏi không có ý nghĩa hoặc không "
        "hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. "
        "Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trả lời rằng bạn không biết và vui lòng không "
        "chia sẻ thông tin sai lệch."
    )

    # Ghép các phần lại với nhau
    formatted_prompt = (
        f"{input_prefix}{pre_prompt_prefix}{pre_prompt}{pre_prompt_suffix}"
        f"Ngữ cảnh: {context}\n\n"
        f"Câu hỏi: {input_text}\n"
        f"{input_suffix}"
    )
    return formatted_prompt

context = """Dấu hiệu và triệu chứng 
Những dấu hiệu và triệu chứng có thể là của ung thư phổi bao gồm:
Triệu chứng về đường hô hấp: ho, ho ra máu, thở khò khè, khó thở
Triệu chứng toàn thân: sụt cân, mệt mỏi, sốt, móng tay dùi trống
Triệu chứng do ung thư chèn ép nhiều sang các cơ quan kề bên: đau ngực, đau xương, tắc nghẽn tĩnh mạch chủ trên, khó nuốt
Nếu ung thư phát triển ở đường thở, nó có thể chặn dòng khí lưu thông, gây ra chứng khó thở. Sự cản trở này có thể dẫn tới việc tích lũy chất bài tiết phía sau chỗ tắc, qua đó mở đường cho viêm phổi.
Phần lớn các triệu chứng của ung thư phổi (chán ăn, sụt cân, sốt, mệt mỏi) là không đặc biệt. Đối với nhiều người, vào thời điểm họ phát hiện ra những dấu hiệu bệnh tật và đi tìm sự chăm sóc y tế, khối u đã lan ra ngoài địa điểm khởi phát. Các triệu chứng có thể báo hiệu quá trình di căn đã xuất hiện bao gồm sụt cân, đau xương và các triệu chứng về thần kinh (đau đầu, ngất xỉu, co giật, yếu chi). Những địa điểm khối u lan sang thường gặp đó là não, xương, tuyến thượng thận, lá phổi còn lại, gan, màng ngoài tim, và thận. Khoảng 10% số ca ung thư phổi không thấy những triệu chứng khi chẩn đoán, những trường hợp này bệnh tình cờ phát hiện nhờ việc chụp X quang ngực định kỳ.
 Nguyên nhân
Triệu chứng
Ung thư phổi là căn bệnh trong đó xuất hiện một khối u ác tính được mô tả qua sự tăng sinh tế bào không thể kiểm soát trong các mô phổi. Nếu người bệnh không được điều trị, sự tăng trưởng tế bào  này có thể lan ra ngoài phổi  đến các mô hoặc bộ phận khác của cơ thể, quá trình này gọi là di căn. Hầu hết các loại ung thư khởi nguồn từ trong phổi (ung thư phổi nguyên phát) là ung thư biểu mô,. Ung thư phổi được chia làm hai loại chính là ung thư phổi tế bào nhỏ (SCLC) và ung thư phổi không phải tế bào nhỏ (NSCLC). Triệu chứng phổ biến nhất của căn bệnh này là ho (bao gồm cả ho ra máu), sụt cân, khó thở, và đau ngực."""
# Run text generation pipeline with our next model
inputs = "Triệu chứng của ung thư phổi?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"{chat_template(inputs, context)}")
print(result[0]['generated_text'])

<s>[INST] <<SYS>>
Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn.
Câu trả lời của bạn không nên chứa bất kỳ nội dung gây hại, phân biệt chủng tộc, phân biệt giới tính, độc hại, nguy hiểm hoặc bất hợp pháp nào. Hãy đảm bảo rằng các câu trả lời của bạn không có thiên kiến xã hội và mang tính tích cực. Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác. Nếu bạn không biết câu trả lời cho một câu hỏi, hãy trả lời rằng bạn không biết và vui lòng không chia sẻ thông tin sai lệch.<</SYS>> 

Ngữ cảnh: Dấu hiệu và triệu chứng 
Những dấu hiệu và triệu chứng có thể là của ung thư phổi bao gồm:
Triệu chứng về đường hô hấp: ho, ho ra máu, thở khò khè, khó thở
Triệu chứng toàn thân: sụt cân, mệt mỏi, sốt, móng tay dùi trống
Triệu chứng do ung thư chèn ép nhiều sang các cơ quan kề bên: đau ngực, đau xương, tắc nghẽn tĩnh mạch chủ trên, kh

In [18]:
# Free VRAM
import gc
del model, pipe, trainer

gc.collect()

0

In [19]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model = PeftModel.from_pretrained(base_model, finetune_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [20]:
model.save_pretrained(PROJECT_PATH + "/model/llm/llm_model.pt")
tokenizer.save_pretrained(PROJECT_PATH + "/model/llm/tokenizer/")


('model/llm/tokenizer/tokenizer_config.json',
 'model/llm/tokenizer/special_tokens_map.json',
 'model/llm/tokenizer/tokenizer.json')